### Cryotherapy dataset

In [1]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_8d1c53da1cbb4d8bb89ec5eaf6508b9e.csv"
df=pd.read_csv(url)
df.head()

,sex,age,Time,Number_of_Warts,Type,Area,Result_of_Treatment
0,1,35,12.00,5,1,100,0
1,1,29,7.00,5,1,96,1
2,1,50,8.00,1,3,132,0
3,1,32,11.75,7,3,750,0
4,1,67,9.25,1,1,42,0


In [2]:
df.shape

(90, 7)

In [3]:
df.groupby("Result_of_Treatment").count().T.head()

Result_of_Treatment,0,1
sex,42,48
age,42,48
Time,42,48
Number_of_Warts,42,48
Type,42,48


In [4]:
X=df.drop(["Result_of_Treatment"],axis=1)
X.head()

,sex,age,Time,Number_of_Warts,Type,Area
0,1,35,12.00,5,1,100
1,1,29,7.00,5,1,96
2,1,50,8.00,1,3,132
3,1,32,11.75,7,3,750
4,1,67,9.25,1,1,42


In [5]:
y=df[["Result_of_Treatment"]]
y.head()

,Result_of_Treatment
0,0
1,1
2,0
3,0
4,0


In [6]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X),columns=X.columns)
X.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


,sex,age,Time,Number_of_Warts,Type,Area
0,-0.956501,0.481695,1.279144,-0.144085,-0.777778,0.108143
1,-0.956501,0.030106,-0.196791,-0.144085,-0.777778,0.077609
2,-0.956501,1.610667,0.098396,-1.271709,1.444444,0.352419
3,-0.956501,0.255900,1.205347,0.419727,1.444444,5.070003
4,-0.956501,2.890170,0.467380,-1.271709,-0.777778,-0.334607


In [7]:
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

forest=RandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=4,min_samples_split=3,min_samples_leaf=1,max_features="auto",bootstrap=True,n_jobs=-1,warm_start=True,class_weight="balanced",random_state=0)
scores=cross_val_score(forest,X,np.asarray(y).reshape(90,),cv=15)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full traini

In [8]:
print("Accuracy: ",scores.mean())

Accuracy:  0.9400000000000001


In [9]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(forest, X, y, cv=15)
AUC = roc_auc_score(y, y_pred)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:850: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/model_selection/_val

In [10]:
print("AUC for the Random Forest model: ",AUC)

AUC for the Random Forest model:  0.9464285714285713


In [11]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n",confusion_matrix(y,y_pred))

Confusion matrix: 
 [[41  1]
 [ 4 44]]


In [69]:
(41+44)/(41+1+4+44)

0.9444444444444444

In [12]:
print("True positive rate: ",44/48)
print("True negative rate: ",41/42)

True positive rate:  0.9166666666666666
True negative rate:  0.9761904761904762


In [13]:
from sklearn.metrics import f1_score
print("F1 score: ",f1_score(y,y_pred))

F1 score:  0.946236559139785


In [14]:
y_prob =pd.DataFrame((cross_val_predict(forest, X, np.asarray(y).reshape(90,), cv=15,method="predict_proba"))).rename(columns={0:"pred_0",1:"pred_1"})

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn('class_weight presets "balanced" or "balanced_subsample" are '
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/ensemble/forest.py:502: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight("balanced", classes, y). In place of y you can use a large enough sample of the full traini

,pred_0,pred_1
0,0.768607,0.231393
1,0.088518,0.911482
2,0.827016,0.172984
3,0.971371,0.028629
4,0.653713,0.346287


In [15]:
y_prob.head()

,pred_0,pred_1
0,0.768607,0.231393
1,0.088518,0.911482
2,0.827016,0.172984
3,0.971371,0.028629
4,0.653713,0.346287


In [16]:
y_probs1=y_prob[["pred_1"]]

In [17]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [18]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [19]:
from sklearn.metrics import f1_score
scores=[f1_score(y, to_labels(y_probs1,t)) for t in thresholds]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jupyte

In [20]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.352, 0.9473684210526315)


In [53]:
y1_pred=round(pd.DataFrame(y_prob).rename(columns={0:"Pred_0",1:"Pred_1"}),3)
y1_pred.head()

,pred_0,pred_1
0,0.769,0.231
1,0.089,0.911
2,0.827,0.173
3,0.971,0.029
4,0.654,0.346


In [54]:
y1_pred["pred_0"]=np.where((y1_pred.pred_0 >=0.352),1,y1_pred.pred_0)
y1_pred["pred_0"]=np.where((y1_pred.pred_0 < 0.352),0,y1_pred.pred_0)
y1_pred["pred_1"]=np.where((y1_pred.pred_1 >=0.352),1,y1_pred.pred_1)
y1_pred["pred_1"]=np.where((y1_pred.pred_1 <0.352),0,y1_pred.pred_1)

In [56]:
y1_pred["pred_0"]=np.where((y1_pred.pred_0 == y1_pred.pred_1),0,y1_pred.pred_0)

In [57]:
len00=len(y1_pred[y1_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  42


In [58]:
len11=len(y1_pred[y1_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  48


In [59]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np
enc=OneHotEncoder(handle_unknown="ignore")
y=np.asarray(y).reshape(-1,1)
y=pd.DataFrame(enc.fit_transform(y).toarray())
y.head()

,0,1
0,1.0,0.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [61]:
print("AUC: ",roc_auc_score(y,y1_pred))

AUC:  0.9330357142857144


In [64]:
print("F1 score: ",f1_score(y,y1_pred,average="macro"))

F1 score:  0.9330357142857143


In [65]:
print("Confusion matrix: \n",np.asarray([[40,2],[2,46]]))

Confusion matrix: 
 [[40  2]
 [ 2 46]]


In [66]:
print("True positive rate: ",46/(46+2))
print("True negative rate: ",40/(40+2))

True positive rate:  0.9583333333333334
True negative rate:  0.9523809523809523


In [70]:
print("Accuracy: ",(40+46)/(40+2+2+46))

Accuracy:  0.9555555555555556
